### All Techniques Of Hyper Parameter Optimization

1. GridSearchCV
2. RandomizedSearchCV
3. Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
4. Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)
4. Optuna- Automate Hyperparameter Tuning
5. Genetic Algorithms (TPOT Classifier)

###### References
- https://github.com/fmfn/BayesianOptimization
- https://github.com/hyperopt/hyperopt
- https://www.jeremyjordan.me/hyperparameter-tuning/
- https://optuna.org/
- https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d(By Pier Paolo Ippolito )
- https://scikit-optimize.github.io/stable/auto_examples/hyperparameter-optimization.html
imization.html


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
#  Laod the dataset
file_path = "diabetes.csv"
df = pd.read_csv(file_path)

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df["Glucose"] = np.where(df["Glucose"] == 0, df["Glucose"].median(), df["Glucose"])

In [8]:
# Split the data into feature and target variable
x = df.drop(columns=["Outcome"])
y = df["Outcome"]

# Split the data into traon and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
# scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [10]:
# convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Ensure the target labels are 1-dimensional
y_train = torch.flatten(y_train)
y_test = torch.flatten(y_test)

In [11]:
# Create DataLoader
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=True)

In [12]:
from skorch import NeuralNetBinaryClassifier
from sklearn.model_selection import RandomizedSearchCV

class Classifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(x_train.shape[1], 64)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(dropout)

        self.fc2 = nn.Linear(64, 32)
        self.batch_norm2 = nn.BatchNorm1d(32)

        self.fc3 = nn.Linear(32, 16)
        self.batch_norm3 = nn.BatchNorm1d(16)

        self.fc4 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)
    
        x = torch.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = torch.relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

criterion = nn.BCELoss()
net = NeuralNetBinaryClassifier(
    Classifier,
    max_epochs=50,
    lr = 0.001,
    optimizer=optim.Adam,
    criterion=criterion,
    iterator_train__shuffle=True,
    verbose=0,
    device=device
)

In [13]:
import os
  
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

Number of CPUs in the system: 28


In [ ]:
param = {
    'lr': [0.01, 0.001, 0.0001],
    'max_epochs': [20, 50, 100],
    'module__dropout': [0.2, 0.3, 0.5],
    'optimizer__weight_decay': [0, 1e-5, 1e-4, 1e-3],
    'batch_size': [32, 64, 128]
}


model = RandomizedSearchCV(
    net,
    param_distributions=param,
    n_iter=200,
    n_jobs=-1,
    random_state=42,
    cv=3,
    verbose=2,
    scoring='accuracy'
)

model.fit(x_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


In [15]:
print("Best Parameters:", model.best_params_)

Best Parameters: {'optimizer__weight_decay': 0.001, 'module__dropout': 0.5, 'max_epochs': 100, 'lr': 0.001, 'batch_size': 64}


In [16]:
best_model = model.best_estimator_
print(best_model)

<class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=Classifier(
    (fc1): Linear(in_features=8, out_features=64, bias=True)
    (batch_norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
    (batch_norm2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc3): Linear(in_features=32, out_features=16, bias=True)
    (batch_norm3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc4): Linear(in_features=16, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  ),
)


In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = best_model.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print("accuracy_score {}".format(accuracy_score(y_test, y_pred)))
print("Classification Report {}".format(classification_report(y_test, y_pred)))

[[81 18]
 [19 36]]
accuracy_score 0.7597402597402597
Classification Report               precision    recall  f1-score   support

         0.0       0.81      0.82      0.81        99
         1.0       0.67      0.65      0.66        55

    accuracy                           0.76       154
   macro avg       0.74      0.74      0.74       154
weighted avg       0.76      0.76      0.76       154

